## Sales Scraper for Loopnet.com

 A scraper dedicated to extracting information from Loopnet.com
 During running a brower will open, it's important that the browser opens with a proper resoltion so that all element are loaded.

## ⚠️WARNING⚠️

Overuse of this script may cause the provider to refuse the connection, either temporarily or permanently. To ensure safe usage:
- Avoid running the script continuously all day.
- Keep the randSleep function (present in the first cell) unchanged or increase its values for longer pauses.
- Only use when necessary
- This scrapper is kinda slow, and it's intented to be slow, as to avoid ban from the provider


## SETUP

1. install the dependecies
2. put the chromedriver on the drivers folder
3. all done


## USAGE

1. Run the first cell to do the setup. A browser will open, go to the desired search page of loopnet.com, something like `https://www.loopnet.com/search/commercial-real-estate/boerne-tx/for-sale/`, no extra random letters/symbosl/numbers at the end. 
2. Do not close the browser.



In [133]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import numpy as np
import random
import json

def randSleep():
    time.sleep(random.uniform(4, 6))


def pretty_print(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key) + ":")
        if isinstance(value, dict):
            pretty_print(value, indent + 1)
        else:
            print("  " * (indent + 1) + str(value))




service = Service("./drivers/chromedriver")
driver = webdriver.Chrome()
driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load
wait = WebDriverWait(driver, 10) # mistery wait

driver.get("https://google.com/")




3. Run the next cell. A browser window will open and navigate through the listings. At the end of the process, a file named `loopnet_links.txt` will be created. This file contains a list of links from which the information will be extracted.

In [95]:
link_list = []
output = open("loopnet_links.txt", "w")

current_page = driver.current_url
print("Current page: ", current_page)

pagenums = driver.find_elements(By.CSS_SELECTOR, ".paging.ng-isolate-scope")[0]
max_page = pagenums.text.split(" ")[-1]
max_page = int(max_page)

for page_num in range(1, max_page+1):
    
    URL = current_page  + "/" + str(page_num)
    if page_num != 1:
        driver.get(URL)
    
    randSleep()
    #placards = driver.find_elements(By.CSS_SELECTOR, ".placard")
    placards = driver.find_elements(By.CSS_SELECTOR, ".placard")
    link_num = 1
    for placard in placards:
        link = placard.find_elements(By.TAG_NAME, "a")
        hrefs = link[0].get_attribute("href")
        link_list.append(hrefs)
        print(page_num,link_num,hrefs)
        output.write(hrefs + "\n")
        link_num += 1
output.close()


print("Total links: ", len(link_list))

Current page:  https://www.loopnet.com/search/commercial-real-estate/boerne-tx/for-lease/
1 1 https://www.loopnet.com/Listing/1580-S-Main-St-Boerne-TX/26030426/
1 2 https://www.loopnet.com/Listing/136-Industrial-Dr-Boerne-TX/33908669/
1 3 https://www.loopnet.com/Listing/206-W-Highland-Dr-Boerne-TX/33616735/
1 4 https://www.loopnet.com/Listing/29-Scenic-Loop-Rd-Boerne-TX/33522963/
1 5 https://www.loopnet.com/Listing/529-E-US-HWY-Boerne-TX/32734039/
1 6 https://www.loopnet.com/Listing/37535-Interstate-10-W-Boerne-TX/32656136/
1 7 https://www.loopnet.com/Listing/1411-S-Main-St-Boerne-TX/32583298/
1 8 https://www.loopnet.com/Listing/720-N-Main-St-Boerne-TX/32583181/
1 9 https://www.loopnet.com/Listing/5-Coughran-Rd-Boerne-TX/31809110/
1 10 https://www.loopnet.com/Listing/IH-10-Fair-Oaks-Blvd-Boerne-TX/31754364/
1 11 https://www.loopnet.com/Listing/9-Spanish-Pass-Rd-Boerne-TX/31569275/
1 12 https://www.loopnet.com/Listing/28615-W-Interstate-10-Boerne-TX/31482243/
1 13 https://www.loopnet.co

4. The next cell is used to load the file that was just saved, helping to avoid redundant runs. If you are not loading from file, e.g. running everthing by sequence, you may skip this cell.

for testing purposes the current file is `loopnet_links_safe.txt` which contains 500 links.

In [120]:
link_list = np.loadtxt("loopnet_links.txt", dtype=str)

print("Total links: ", len(link_list))

Total links:  41


5. The following cell opens each link from the list in the file, extracts all relevant information from each one, and saves the data into a `.JSON` file named `loopnet_data.json`. You may also change how many links are scraped by changing `init_page` and `final_page`, if one wishes to do scrapping by parts. When running this part, a new window will appear, it needs to be `1980x1080` to ensure that all properties of the page are properly loaded.
   
Remember, here 0 index corresponds to the 1st page, 1 to the 2nd and so on.

In [134]:
# Now the proper data scrapping
init_page = 0
final_page = 10

# DONE HERE
def get_title(): # Tittle info OK
    title = driver.find_elements(By.TAG_NAME, "h1")
    title = title[0].text # INFO
    return title

# DONE HERE
def get_footnote(): # footnoot info, aways present
    footnote_infos = driver.find_elements(By.CLASS_NAME, "property-timestamp")
    footnote_infos = footnote_infos[0].find_elements(By.TAG_NAME, "h4")
    listing_id = footnote_infos[0].text.split(":")[-1] # INFO
    market_date = footnote_infos[1].text.split(":")[-1] # INFO
    last_update = footnote_infos[2].text.split(":")[-1] # INFO
    address = footnote_infos[3].text.split(":")[-1] # INFO
    return listing_id, market_date, last_update, address


# DONE HERE?
def get_facts(): # facts info (optional)
    facts = driver.find_elements(By.CSS_SELECTOR, ".listing-features")
    if len(facts) == 0:
        return {}
    
    facts_names = facts[0].find_elements(By.CSS_SELECTOR, ".fact-name")
    vals = facts[0].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")
    dic_facts = {}
    for i in range(0, len(facts_names)):
        dic_facts[facts_names[i].text] = vals[i].text

    return dic_facts

def get_amenites():
    amenites_all = driver.find_elements(By.CSS_SELECTOR, ".highlights.features-and-amenities.listing-features_amenities")
    features = []
    if len(amenites_all) != 0:
        for list_part in amenites_all:
            amenities = list_part.find_elements(By.TAG_NAME, "li")
            for amenity in amenities:
                features.append(amenity.text)
    features = ", ".join(features)
    return features

# DONE HERE
def get_tax():
    tax = {}
    tax_all = driver.find_elements(By.ID, "taxes")
    if len(tax_all) == 0:
        return tax
    infos = tax_all[0].find_elements(By.TAG_NAME, "td")
    for i in range(0, len(infos)):
        if i % 2 == 0:
            tax[infos[i].text] = infos[i+1].text
    return tax


# DONE HERE
def get_zoning():
    zoning = {}
    zoning_all = driver.find_elements(By.CSS_SELECTOR, ".taxes-zoning.taxes-zoning__zoning")
    if len(zoning_all) == 0:
        return zoning
    infos = zoning_all[0].find_elements(By.TAG_NAME, "td")
    for i in range(0, len(infos)):
        if i % 2 == 0:
            zoning[infos[i].text] = infos[i+1].text
    return zoning

def get_demographics1():
    demographics = []
    demographics_all = driver.find_elements(By.CSS_SELECTOR, ".demographics-industrial__table-one")
    if len(demographics_all) == 0:
        return demographics
    else:
        infos = demographics_all[0].find_elements(By.CLASS_NAME, "trade-area-table")
        for i in range(1, len(infos)):
            demographics.append(infos[i].text.replace("\n", " "))
    return demographics

# DONE HERE
def get_brokers():
    names = []
    phones = []
    brokers_infos = driver.find_elements(By.CSS_SELECTOR, ".broker-bio") # INFO
    if brokers_infos != []:
        for i in range(len(brokers_infos)):
            name = brokers_infos[i].find_elements(By.CLASS_NAME, "header-text")[0].text
            phone = brokers_infos[i].find_elements(By.CLASS_NAME, "broker-phone")[0].text
            if phone == "":
                phone = "Not available"
            else:
                phones.append(phone)
            names.append(name)

    else:
        brokers_infos = driver.find_element(By.CSS_SELECTOR, ".container-contact-form ")
        name = brokers_infos.find_elements(By.CSS_SELECTOR, ".contact-name")
        name = name[0].text
        phone = brokers_infos.find_elements(By.CSS_SELECTOR, ".phone-number")
        if phone == []:
            phone = "Not available"
        else:
            phones.append(phone[0].text)
        names.append(name)

    return names, phones

def get_labor():
    demographics = {}
    demographics_all = driver.find_elements(By.CSS_SELECTOR, ".demographics-industrial__table-two")
    if len(demographics_all) == 0:
        return demographics
    
    else:
        row_names = demographics_all[0].find_elements(By.CLASS_NAME, "row-label")
        table_data = demographics_all[0].find_elements(By.CLASS_NAME, "table-data")
        for i in range(1, len(row_names)):
            demographics[row_names[i].text] = table_data[i].text
    return demographics

def get_nearby_am(): # need a quick fix
    # check for hotels:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    pois = {}
    near_interests = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.poi ")
    if len(near_interests) == 0:
        return []

    for poi in near_interests:
        name = poi.find_elements(By.TAG_NAME, "h3")
        places = []
        for place in poi.find_elements(By.TAG_NAME, "tr"):
            places.append(place.text)
        pois[name[0].text] = places[1:]
    return pois

def get_utilities():
    utilities = []
    utilities_all = driver.find_elements(By.CSS_SELECTOR, ".highlights.include-in-page.utilities")
    if len(utilities_all) == 0:
        return utilities
    infos = utilities_all[0].find_elements(By.TAG_NAME, "li")
    for i in range(0, len(infos)):
        utilities.append(infos[i].text)
    
    return utilities

def get_spaces():
    ## Fix to add zoning
    spaces = {}
    spaces_table = driver.find_elements(By.CSS_SELECTOR, ".available-spaces.available-spaces--desktop.available-spaces--cards")
    print(len(spaces_table))
    if len(spaces_table) > 0:
        for i in range(len(spaces_table)):
            #print(i,len(spaces_table))
            this_space = {}
            this_space['address_space'] = spaces_table[i].find_elements(By.CSS_SELECTOR, ".as-space-info__address")[0].text
            fact_names = spaces_table[i].find_elements(By.CSS_SELECTOR, ".fact-name")
            fact_vals = spaces_table[i].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")
            for j in range(len(fact_names)):
                this_space[fact_names[j].text] = fact_vals[j].text
            this_space["zooning_space"] = spaces_table[i].find_elements(By.CSS_SELECTOR, ".property-facts__data-item-text ")[-1].text


            # get basic property of the listing
            floors_basic_prop = spaces_table[i].find_elements(By.CSS_SELECTOR, ".available-spaces__header-headings")
            floors_basic_prop = floors_basic_prop[0].text.split("\n")

            floors = spaces_table[i].find_elements(By.CSS_SELECTOR, ".available-spaces__content")
            floors_basic_vals = floors[0].text.split("\n")
            print(floors_basic_vals)

            for j in range(0,len(floors_basic_prop)):
                this_space[floors_basic_prop[j]] = floors_basic_vals[j]       

            for j in range(0,len(floors)):
                floors[j].click()
                time.sleep(1)
                extras = floors[j].find_elements(By.CSS_SELECTOR, ".available-spaces__highlights-wrap")
                if extras != []:
                    extras = extras[0].text.split("\n")
                    extras = ", ".join(extras)
                    this_space["facts_space"] = extras
            
            spaces["space_" + str(i)] = this_space

        return spaces

    else:
        print("No cards")
        this_space = {}

        floors = driver.find_elements(By.CSS_SELECTOR, ".available-spaces__content")
        # get basic property of the listing
        
        floors_basic_prop = driver.find_elements(By.CSS_SELECTOR, ".available-spaces__header")
        if len(floors_basic_prop) == 0:
            return spaces
        
        floors_basic_prop = floors_basic_prop[0].text.split("\n")

        floors = driver.find_elements(By.CSS_SELECTOR, ".available-spaces__content")
        floors_basic_vals = floors[0].text.split("\n")
        print(floors_basic_vals)

        for j in range(0,len(floors_basic_prop)):
            this_space[floors_basic_prop[j]] = floors_basic_vals[j]       

        for j in range(0,len(floors)):
            floors[j].click()
            time.sleep(1)
            extras = floors[j].find_elements(By.CSS_SELECTOR, ".available-spaces__highlights-wrap")
            if extras != []:
                extras = extras[0].text.split("\n")
                extras = ", ".join(extras)
                this_space["facts_space"] = extras

            spaces["space_" + str(j)] = this_space
        return spaces


print("Total links: ", len(link_list))
print("Links:")


driver.set_window_size(1980, 1080)

all_data = []
for link in range(init_page,len(link_list)):
    if init_page == final_page:
        break
    data = {}
    print("c: ", init_page, link_list[link])
    driver.get(link_list[link])
    randSleep()

    data["link"] = link_list[link] # link for the ad
    data["title"] = get_title() # title of the ad
    data["brokers"], data["phones"] = get_brokers() # broker info
    list_id, market_date, last_update, address = get_footnote()
    data["list ID"] = list_id # listing ID
    data["market date"] = market_date # market date
    data["last update"] = last_update # last update
    data["address"] = address # address
    data["facts"] = get_facts() # facts
    data["amenities"] = get_amenites() # amenities
    data["utilities"] = get_utilities() # utilities
    data["tax"] = get_tax() # tax
    data["zoning"] = get_zoning() # zoning
    data["spaces"] = get_spaces() # spaces (it will create lots of extra fields on the table)
    data["labor"] = get_labor() # labor
    data["demographics"] = get_demographics1() # demographics
    data["poi"] = get_nearby_am() # nearby amenities
    all_data.append(data)
    print("Data:")
    pretty_print(data)

    print("Not available")
    init_page +=1

print("Done \a")

output = open("loopnet_data.json", "w")
json.dump(all_data, output, indent=4)
output.close()
    

Total links:  41
Links:
c:  0 https://www.loopnet.com/Listing/1580-S-Main-St-Boerne-TX/26030426/
0
No cards
['1st Floor, Ste 100', '2,241 SF', 'Negotiable', '$25.00 /SF/YR', 'Office/Medical', '-', '30 Days']
Data:
link:
  https://www.loopnet.com/Listing/1580-S-Main-St-Boerne-TX/26030426/
title:
  Boerne Gateway 1580 S Main St
brokers:
  ['Lindsey Tucker, Partner', 'Amanda Powell']
phones:
  ['+1 210-876-2240', '+1 281-733-9545']
list ID:
   26030426
market date:
   10/29/2024
last update:
   11/26/2024
address:
   1580 S Main St, Boerne, TX 78006
facts:
  Building Type:
    Office
  Year Built:
    2000
  Building Height:
    2 Stories
  Building Size:
    31,449 SF
  Building Class:
    B
  Typical Floor Size:
    15,725 SF
  Unfinished Ceiling Height:
    12’
amenities:
  
utilities:
  []
tax:
zoning:
spaces:
  space_0:
    SPACE:
      1st Floor, Ste 100
    SIZE:
      2,241 SF
    TERM:
      Negotiable
    RENTAL RATE:
      $25.00 /SF/YR
    SPACE USE:
      Office/Medical
    C

6. The following cell converts the `.JSON` data into a table (or DataFrame) containing all the extracted information, without filtering by attributes. It also prints all the possible fields that may be of interest for saving. If a field is blank, the information was not available.

In [115]:
# Table generator
import pandas as pd

def print_all_keys(d, prefix=""):
    if isinstance(d, dict):
        for key in d.keys():
            print(f"{prefix}{key}")
            print_all_keys(d[key], prefix + "  ")  # Indent for nested keys
    elif isinstance(d, list):
        for item in d:
            print_all_keys(item, prefix)

def flatten_json(json_obj, parent_key="", sep="_"):
    items = []
    if isinstance(json_obj, dict):
        for k, v in json_obj.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_json(v, new_key, sep=sep).items())
    elif isinstance(json_obj, list):
        for i, v in enumerate(json_obj):
            new_key = f"{parent_key}{sep}{i}" if parent_key else str(i)
            items.extend(flatten_json(v, new_key, sep=sep).items())
    else:
        items.append((parent_key, json_obj))
    return dict(items)

all_data = json.load(open("loopnet_data.json"))


df = pd.json_normalize(all_data)
df = df.applymap(lambda x: ", ".join(x) if isinstance(x, list) else x)

df.to_excel("all_loopnet_data.xlsx", index=False)

print("Available infos:")
print_all_keys(all_data)


for field in field_list:
    print(field)




Available infos:
link
title
spaces
  space_0
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_1
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_2
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_3
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_4
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_5
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
  space_6
    SPACE
    SIZE
    TERM
    RENTAL RATE
    SPACE USE
    CONDITION
    AVAILABLE
    facts_space
brokers
phones
facts
amenities
utilities
demographics
labor
poi
  RESTAURANTS
  RETAIL
tax
zoning
list ID
market date
last update
address
link
title
spaces
  space_0
    SPACE
    S

With all the data scraped and secure, this next cell servers to save a filtered table, if the one may desire.

7. To do so, change the words on the array `my_fields`, so it may look something like `my_fields = ["link", "title", "brokers.names", "brokers.phones"]`.
8. Run the cell, and a new excel file with name `filtered_loopnet_data.xlsx` will appear, this one contais only the filtered data defined by `my_fields`

In [118]:
my_fields = ["link", "title","brokers","poi"]

df_to_save = df[my_fields]
print("Filtered infos:")
print(df_to_save)
df_to_save.to_excel("filtered_loopnet_data.xlsx", index=False)


Filtered infos:
                                                 link  \
0   https://www.loopnet.com/Listing/1580-S-Main-St...   
1   https://www.loopnet.com/Listing/136-Industrial...   
2   https://www.loopnet.com/Listing/206-W-Highland...   
3   https://www.loopnet.com/Listing/29-Scenic-Loop...   
4   https://www.loopnet.com/Listing/529-E-US-HWY-B...   
5   https://www.loopnet.com/Listing/37535-Intersta...   
6   https://www.loopnet.com/Listing/1411-S-Main-St...   
7   https://www.loopnet.com/Listing/720-N-Main-St-...   
8   https://www.loopnet.com/Listing/5-Coughran-Rd-...   
9   https://www.loopnet.com/Listing/IH-10-Fair-Oak...   
10  https://www.loopnet.com/Listing/9-Spanish-Pass...   
11  https://www.loopnet.com/Listing/28615-W-Inters...   
12  https://www.loopnet.com/Listing/1201-S-Main-St...   
13  https://www.loopnet.com/Listing/1415-E-Blanco-...   
14  https://www.loopnet.com/Listing/312-River-Rd-B...   
15  https://www.loopnet.com/Listing/38-Rust-Ln-Boe...   
16  https://www